In [ ]:
from flask import Flask, url_for, request, jsonify, redirect, Response
from flask_cors import CORS, cross_origin

import json
import requests

In [ ]:
KAKAO_TOKEN_SAVED_FILE = "res/kakao_msg/kakao_token.json"
APP_KEY_SAVED_FILE = "res/kakao_msg/app_key"
NAVER_USER_DATA_SAVED_FILE = "res/naver/accout"

In [ ]:
# 네이버 계정 정보를 저장하기 위해 최초로 한 번만 동작시키도록 한다.
# 이후로는 또 동작시켜서는 안된다(계정 정보가 날아갈테니까)
import pickle

naver_user_info = dict({
    "smtp_user_id": "사용자계정@naver.com",
    "smtp_user_pw": "비밀번호"
})

f = open(NAVER_USER_DATA_SAVED_FILE, 'wb')
pickle.dump(naver_user_info, f)
f.close()

In [ ]:
# KAKAO REST API KEY 정보를 저장하기 위해 최초로 한 번만 동작시키도록 한다.
# 이후로는 또 동작시켜서는 안된다(키값이 날아갈테니까)
import pickle

app_key = ""

f = open(APP_KEY_SAVED_FILE, 'wb')
pickle.dump(app_key, f)
f.close()

In [ ]:
# REST APP KEY 저장 잘 되었는지 확인하는 목적
import pickle

f = open(APP_KEY_SAVED_FILE, 'rb')
app_key = pickle.load(f)
f.close()

In [ ]:
# NAVER 계정 정보를 가져옴
import pickle

f = open(NAVER_USER_DATA_SAVED_FILE, 'rb')
naver_user_info = pickle.load(f)
f.close()

print()

In [ ]:

# 저장
def save_tokens(filename, tokens):
    with open(filename, "w") as fp:
        json.dump(tokens, fp)

In [ ]:
# 토큰 만료시 재발급
import requests

url = "https://kauth.kakao.com/oauth/token"

#data = {
#    "grant_type" : "authorization_code",
#    "client_id" : "REST_API_키",
#    "redirect_uri" : "https://localhost.com",
#    "code" : "발급받은 토큰값"
#}

data = {
    "grant_type" : "authorization_code",
    "client_id" : app_key,
    "redirect_uri" : "https://localhost.com",
    "code" : ""
}

response = requests.post(url, data=data)

if response.status_code != 200:
    print("Error!")
else:
    tokens = response.json()
    print(tokens)
    
# token 저장
save_tokens(KAKAO_TOKEN_SAVED_FILE, tokens)

In [ ]:
def load_tokens(filename):
    with open(filename) as fp:
        tokens = json.load(fp)
        
    return tokens

In [ ]:

import os
import smtplib

from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
from email.mime.audio import MIMEAudio
from email.mime.text import MIMEText
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

In [ ]:
def send_email(smtp_info, naver_user_info, msg):
    with smtplib.SMTP(smtp_info["smtp_server"], smtp_info["smtp_port"]) as server:
        # TLS 보안 설정 및 시작 (보안 정책상 필수)
        server.starttls()
        
        # 로그인
        server.login(naver_user_info["smtp_user_id"], naver_user_info["smtp_user_pw"])
        
        # 이메일 전송
        # 메시지 전송시 반드시 as_string()으로 변환하여 전송해야함
        res = server.sendmail(msg['from'], msg['to'], msg.as_string())
        
        if not res:
            print('이메일 전송 성공!!!')
        else:
            print(res)

In [ ]:
def make_multimsg(msg_dict):
    multi = MIMEMultipart(_subtype = 'mixed')
    
    for key, value in msg_dict.items():
        # text 형식인 경우
        if key == 'text':
            with open(value['filename'], encoding='utf-8') as fp:
                msg = MIMEText(fp.read(), _subtype = value['subtype'])
        # image 형식인 경우
        elif key == 'image':
            with open(value['filename'], 'rb') as fp:
                msg = MIMEImage(fp.read(), _subtype = value['subtype'])
        # audio, video는 별도 설정 없이 진행하도록 함
        elif key == 'audio':
            print("음원은 지원하지 않습니다!")
        elif key == 'video':
            print("영상은 지원하지 않습니다!")
        # 그 외의 형식
        else:
            with open(value['filename'], 'rb') as fp:
                msg = MIMEBase(value['maintype'], _subtype = value['subtype'])
                msg.set_payload(fp.read())
                encoders.encode_base64(msg)
                
        # 파일 이름은 첨부파일 제목
        msg.add_header('Content-Disposition', 'attachment', 
                       filename = os.path.basename(value['filename']))

        # 파일 첨부!
        multi.attach(msg)
        
    return multi

In [ ]:
import datetime

app = Flask(__name__)
CORS(app)

@app.route('/spring2pythonRequestWithData', methods = ['GET', 'POST'])
def spring2pythonRequestWithData():
    
    params = json.loads(request.get_data(), encoding='utf-8')
    print("params: ", params)
    
    if len(params) == 0:
        return jsonify("No Parameter")
    
    params_string = ""
    customer_name = ""
    price = ""
    
    ##################### 개조 구간 시작 #####################
    #params_string = ""
    #for key in params.keys():
    #    params_string += 'key: {}, value: {}<br>'.format(key, params[key])
    
    values = []
    for key in params.keys():
        values.append(params[key])

    print("values: ", values)
    customer_name = values[0]
    price = values[1]
    #imgPath = values[2]
    
    print("이름: ", customer_name)
    print("가격: ", price)
    #print("이미지 경로: ", imgPath)
    
    tokens = load_tokens(KAKAO_TOKEN_SAVED_FILE)
    #print("tokens: ", tokens)

    url = "https://kapi.kakao.com/v2/api/talk/memo/default/send"

    headers = {
        "Authorization" : "Bearer " + tokens['access_token']
    }
    
    message = "주문 테스트 - 구매자: {0}, 결제 금액: {1}".format(customer_name, price)

    data = {
        "template_object" : json.dumps({
            "object_type" : "text",
            "text" : message,
            "link" : {
                "web_url" : "http://localhost:8080/"
            }
        })
    }

    response = requests.post(url, headers=headers, data=data)
    print(response.status_code)
    
    #if response.status_code == 401:
    #    tokens = update_tokens(app_key, KAKAO_TOKEN_SAVED_FILE)
        
    #print("tokens: ", tokens)
    
    ##################### 개조 구간 끄읕 #####################
    
    ##################### 이메일 전송 시작 #####################
    
    smtp_info = dict({
        "smtp_server": "smtp.naver.com",
        "smtp_port": 587
    })

    msg_dict = {
        # 텍스트 형식의 첨부 파일
        #'text': {
        #    'maintype': 'text',
        #    'subtype': 'plain',
            #'filename': 'res/email/test.txt'
        #},
        # 이미지 형식의 첨부 파일
        'image': {
            'maintype': 'image',
            'subtype': 'jpg',
            #'filename': 'res/email/bonobono.jpg'
        },
        # 세금계산서 등등의 pdf 형식의 첨부 파일들
        #'application': {
        #    'maintype': 'application',
        #    'subtype': 'octect-stream',
            #'filename': 'res/email/test.pdf'
        #}
    }
    
    # 메일 작성
    title = "첨부파일이 날아간드아!"
    content = "ㅇㅇ 이건 메일 내용이야"
    sender = naver_user_info['smtp_user_id']
    receiver = naver_user_info['smtp_user_id']
    
    # 결제 사항 적용
    
    now = datetime.datetime.now()
    now_time = now.strftime('%Y-%m-%d %H:%M:%S')
    print(now_time)
    
    title = customer_name + "님 " + now_time + " 결제하신 내역입니다."
    content = "구매하신 물품은 뭣이며 금액은 " + str(price) + "원 입니다. 앞으로도 저희 매장 많이 찾아주세요 ^^"
    
    ##############################

    msg = MIMEText(_text = content, _charset = 'utf-8')
    
    print(msg_dict)
    print(msg_dict['image'])
    print(msg_dict['image']['maintype'])
    
    # 특정 위치 지정하기
    msg_dict['image']['filename'] = 'res/email/bonobono.jpg'
    
    print(msg_dict['image'])

    # 파일 첨부하기!
    multi = make_multimsg(msg_dict)
    multi['Subject'] = title
    multi['From'] = sender
    multi['To'] = receiver
    multi.attach(msg)

    # 첨부한 내용을 추가하여 이메일 전송!
    send_email(smtp_info, naver_user_info, multi)

    ##################### 이메일 전송 끄읕 #####################
    
    return jsonify(params_string)

if __name__ == "__main__":
    app.run()